In [3]:
import sys
import time
import json
from pathlib import Path
# # 공통 설정 및 shared 모듈 경로 추가
root_path = Path.cwd().parent.parent  
sys.path.insert(0, str(root_path))
sys.path.insert(0, str(root_path / "shared"))
from bedrock_agentcore_starter_toolkit import Runtime
from runtime_utils import create_agentcore_runtime_role
import boto3
from boto3.session import Session


In [4]:
def store_agent_info_to_ssm(launch_result):
    print("deployment 정보 저장")
    print(launch_result)
    boto_session = Session()
    ssm_client = boto_session.client('ssm', region_name=launch_result["region"])

    """Store agent information to SSM Parameter Store"""
    ssm_client.put_parameter(
        Name='/rag_server/runtime_iam/agent_arn',
        Value=launch_result["agent_arn"],
        Type='String',
        Description='Agent ARN for MCP server',
        Overwrite=True
    )
    
    ssm_client.put_parameter(
        Name='/rag_server/runtime_iam/agent_id',
        Value=launch_result["agent_id"],
        Type='String',
        Description='Agent ID for MCP server',
        Overwrite=True
    )
    
    ssm_client.put_parameter(
        Name='/rag_server/runtime_iam/execution_role_arn',
        Value=launch_result["iam_role_name"],
        Type='String',
        Description='Execution Role ARN for MCP server',
        Overwrite=True
    )

    ssm_client.put_parameter(
        Name='/rag_server/runtime_iam/ecr_repository_uri',
        Value=launch_result["ecr_repo_url"],
        Type='String',
        Description='ECR Repository URI for MCP server',
        Overwrite=True
    )

In [6]:
# Regions Setting
region_name = boto3.Session().region_name
# Agentic Core 이름 setting
AGENT_NAME = "Rag_Agentic_Core"

In [7]:
# AgenticCore가 사용할 권한 생성
iam_role = create_agentcore_runtime_role(AGENT_NAME, region_name)
iam_role_name = iam_role['Role']['RoleName']


🔐 Runtime IAM 역할 생성 중...
✅ 새 IAM 역할 생성 완료
✅ 권한 정책 연결 완료


In [8]:
#AgenticCore가 Runtime 구성
current_dir = Path.cwd()
runtime = Runtime()
runtime.configure(
    entrypoint=str(current_dir / "rag_bedrockCore.py"),
    execution_role=iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file=str(current_dir / "requirements.txt"),
    region=region_name, 
    agent_name=AGENT_NAME
)

Entrypoint parsed: file=/home/sagemaker-user/smus_agentic_core/agentic_core/code/rag_agentic_core/rag_agent/rag_bedrockCore.py, bedrock_agentcore_name=rag_bedrockCore
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: Rag_Agentic_Core
Memory disabled
Network mode: PUBLIC


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/smus_agentic_core/agentic_core/code/rag_agentic_core/rag_agent/.dockerignore
Setting 'Rag_Agentic_Core' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/smus_agentic_core/agentic_core/code/rag_agentic_core/rag_agent/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/sagemaker-user/smus_agentic_core/agentic_core/code/rag_agentic_core/rag_agent/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/sagemaker-user/smus_agentic_core/agentic_core/code/rag_agentic_core/rag_agent/Dockerfile'), dockerignore_path=PosixPath('/home/sagemaker-user/smus_agentic_core/agentic_core/code/rag_agentic_core/rag_agent/.dockerignore'), runtime='Docker', runtime_type=None, region='us-west-2', account_id='967607059098', execution_role='arn:aws:iam::967607059098:role/agentcore-runtime-Rag_Agentic_Core-role', ecr_repository=None, auto_create_ecr=True, s3_path=None, auto_create_s3=False, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

In [9]:
# Agentic Core 배포
launch_result = runtime.launch(auto_update_on_conflict=True)


🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'Rag_Agentic_Core' to account 967607059098 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: Rag_Agentic_Core
ECR repository available: 967607059098.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-rag_agentic_core
Using execution role from config: arn:aws:iam::967607059098:role/agentcore-runtime-Rag_Agentic_Core-role
Preparing CodeBuild project and uploading source...


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-rag_agentic_core


Getting or creating CodeBuild execution role for agent: Rag_Agentic_Core
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-26b9b491f2
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-26b9b491f2
Creating IAM role: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-26b9b491f2
✓ Role created: arn:aws:iam::967607059098:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-26b9b491f2
Attaching inline policy: CodeBuildExecutionPolicy to role: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-26b9b491f2
✓ Policy attached: CodeBuildExecutionPolicy
Waiting for IAM role propagation...
CodeBuild execution role creation complete: arn:aws:iam::967607059098:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-26b9b491f2
Using dockerignore.template with 43 patterns for zip filtering
Uploaded source to S3: Rag_Agentic_Core/source.zip
Created CodeBuild project: bedrock-agentcore-rag_agentic_core-builder
Starting CodeBuild build (this may take several minutes)...
Startin

In [10]:
# Agentic Core 배포 상태 확인
for i in range(30):
    try:
        status = runtime.status().endpoint['status']
        print(f"📊 상태: {status} ({i*30}초 경과)")
        if status in ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']:
            break
    except Exception as e:
        print(f"⚠️ 상태 확인 오류: {e}")
    time.sleep(30)

if status != 'READY':
    raise Exception(f"배포 실패: {status}")

# ECR 리포지토리 이름 추출
ecr_repo_name = None
if hasattr(launch_result, 'ecr_uri') and launch_result.ecr_uri:
    ecr_repo_name = launch_result.ecr_uri.split('/')[-1].split(':')[0]

Retrieved Bedrock AgentCore status for: Rag_Agentic_Core


📊 상태: READY (0초 경과)


In [11]:
# 베포 정보 정리
deployment_info = {
        "agent_arn": launch_result.agent_arn,
        "agent_id": launch_result.agent_id,
        "region": region_name,
        "iam_role_name": iam_role["Role"]["Arn"],
        "ecr_repo_url": launch_result.ecr_uri
    }

In [12]:
#배포 정보 SSM에 저장 
store_agent_info_to_ssm(deployment_info )

deployment 정보 저장
{'agent_arn': 'arn:aws:bedrock-agentcore:us-west-2:967607059098:runtime/Rag_Agentic_Core-aELDB2Ahea', 'agent_id': 'Rag_Agentic_Core-aELDB2Ahea', 'region': 'us-west-2', 'iam_role_name': 'arn:aws:iam::967607059098:role/agentcore-runtime-Rag_Agentic_Core-role', 'ecr_repo_url': '967607059098.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-rag_agentic_core'}
